This notebook is using transfer learning from a ViT model from huggingface


In [1]:
import multiprocessing
import transformers
import torch
import wandb
import numpy as np
import pytorch_lightning as pl

from torchvision import transforms
from torchvision.datasets import Food101
from torchvision.datasets.utils import download_url
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

from datasets import load_metric, load_dataset
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from transformers import AutoFeatureExtractor
from transformers import DefaultDataCollator

In [3]:
metric = load_metric("accuracy")
food = load_dataset("food101", split="train[:5000]")

splits = food.train_test_split(test_size=0.15)
train = splits['train']
val = splits['test']

display(train.shape, val.shape)

Using custom data configuration default
Reusing dataset food101 (C:\Users\truon\.cache\huggingface\datasets\food101\default\0.0.0\7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295)


(4250, 2)

(750, 2)

In [4]:
# Process image to tensor using ViT method (16x16 patches)
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}

The `AutoFeatureExtractor.from_pretrained()` method helps us make sure we are 
- (1) resizing the inputs to the appropriate size 
- (2) using the appropriate image mean and standard deviation for the model architecture we are going to use

In [5]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_transforms = Compose(
    [
            RandomResizedCrop(feature_extractor.size)
            ,ToTensor()
            ,normalize
    ]
)

val_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

# when the data is loaded, it will apply the transformation above
train = train.with_transform(transforms)
val = val.with_transform(transforms)

# example of how our data is organized, dictionary with 2 key-value pairs
train[0]


{'label': 6,
 'pixel_values': tensor([[[-0.1137, -0.1373, -0.1216,  ...,  0.3961,  0.3725,  0.0824],
          [-0.0902, -0.1216, -0.1529,  ...,  0.4039,  0.1843, -0.0980],
          [-0.0667, -0.1059, -0.1608,  ...,  0.4745,  0.2000, -0.0431],
          ...,
          [-0.9608, -0.9765, -0.9843,  ...,  0.3412,  0.4510,  0.3882],
          [-0.9529, -0.9686, -0.9765,  ...,  0.5216,  0.5843,  0.5059],
          [-0.9451, -0.9608, -0.9608,  ...,  0.6549,  0.6392,  0.5765]],
 
         [[ 0.1451,  0.1373,  0.1529,  ...,  0.6549,  0.6235,  0.3333],
          [ 0.1765,  0.1529,  0.1137,  ...,  0.6784,  0.4510,  0.1765],
          [ 0.2078,  0.1529,  0.0824,  ...,  0.7569,  0.4824,  0.2392],
          ...,
          [-0.9529, -0.9608, -0.9686,  ...,  0.5922,  0.7020,  0.6392],
          [-0.9451, -0.9608, -0.9608,  ...,  0.7569,  0.8196,  0.7412],
          [-0.9373, -0.9529, -0.9529,  ...,  0.8902,  0.8667,  0.8039]],
 
         [[ 0.3176,  0.3098,  0.3255,  ...,  0.7569,  0.7255,  0.4353],

In [6]:
# 3 RGBs, 224 x 224 pixels
train[0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [7]:
# ~ creating a dataloader, creates batches 
# `pt` is for PyTorch Tensor
data_collator = DefaultDataCollator()
data_collator

DefaultDataCollator(return_tensors='pt')

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
labels = train.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

model = AutoModelForImageClassification.from_pretrained(
    pretrained_model_name_or_path="google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb",
    #push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics = compute_metrics,
    tokenizer=feature_extractor,
)


PyTorch: setting up devices
Using cuda_amp half precision backend


In [15]:
train_results = trainer.train()
trainer.save_model(output_dir = "./models/")
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\truon\anaconda3\envs\learn\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4250
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1330
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


  0%|          | 0/1330 [00:00<?, ?it/s]

{'loss': 0.3071, 'learning_rate': 0.00019849624060150375, 'epoch': 0.08}
{'loss': 0.2252, 'learning_rate': 0.00019699248120300754, 'epoch': 0.15}
{'loss': 0.1418, 'learning_rate': 0.00019548872180451127, 'epoch': 0.23}
{'loss': 0.2298, 'learning_rate': 0.00019398496240601503, 'epoch': 0.3}
{'loss': 0.1833, 'learning_rate': 0.0001924812030075188, 'epoch': 0.38}
{'loss': 0.1479, 'learning_rate': 0.00019097744360902256, 'epoch': 0.45}
{'loss': 0.1387, 'learning_rate': 0.00018947368421052632, 'epoch': 0.53}
{'loss': 0.1216, 'learning_rate': 0.00018796992481203009, 'epoch': 0.6}
{'loss': 0.1127, 'learning_rate': 0.00018646616541353382, 'epoch': 0.68}
{'loss': 0.1442, 'learning_rate': 0.0001849624060150376, 'epoch': 0.75}
{'loss': 0.2092, 'learning_rate': 0.00018345864661654135, 'epoch': 0.83}
{'loss': 0.0971, 'learning_rate': 0.0001819548872180451, 'epoch': 0.9}
{'loss': 0.2686, 'learning_rate': 0.00018045112781954887, 'epoch': 0.98}


***** Running Evaluation *****
  Num examples = 750
  Batch size = 8


  0%|          | 0/94 [00:00<?, ?it/s]

...

{'eval_loss': 0.2562807500362396, 'eval_accuracy': 0.9306666666666666, 'eval_runtime': 7.0987, 'eval_samples_per_second': 105.652, 'eval_steps_per_second': 13.242, 'epoch': 10.0}


Model weights saved in ./results\checkpoint-1330\pytorch_model.bin
Feature extractor saved in ./results\checkpoint-1330\preprocessor_config.json
Deleting older checkpoint [results\checkpoint-1197] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-1064 (score: 0.9306666666666666).
Saving model checkpoint to ./models/
Configuration saved in ./models/config.json


{'train_runtime': 729.4853, 'train_samples_per_second': 58.26, 'train_steps_per_second': 1.823, 'train_loss': 0.12560736727445646, 'epoch': 10.0}


Model weights saved in ./models/pytorch_model.bin
Feature extractor saved in ./models/preprocessor_config.json


***** train metrics *****
  epoch                    =       10.0
  train_loss               =     0.1256
  train_runtime            = 0:12:09.48
  train_samples_per_second =      58.26
  train_steps_per_second   =      1.823
